# Exploring & Creating Tables, EIA Selected Fuels

In [85]:
import pandas as pd
import sqlalchemy
import pymysql
from sqlalchemy import create_engine
from sqlalchemy import text
import requests
import datetime as dt

In [285]:
with open('db_string', 'r') as cadena:
    engine = create_engine(cadena.readline())

In [3]:
with open('API_KEY', 'r') as cadena:
    api_key = cadena.readline().strip()

#### Tables:  
EIA_Fuels 
+ OilProd ✓  
+ OilPrice ✓  
+ NGasPrice ✓  
+ NGasStorage ✓
+ DryGasProd ✓  
+ STEO ✓  
+ APIStrings ✓  
+ MetaOilProd ✓  
+ MetaOilPrice ✓  
+ MetaNGasPrice ✓  
+ MetaNGasStorage ✓
+ MetaDryGasProd ✓  
+ MetaSTEO ✓ 




## OilProd & MetaOilProd

In [4]:
url_OilProd = 'https://api.eia.gov/v2/petroleum/crd/crpdn/data/?frequency=monthly&data[0]=value&facets[series][]=MCRFPP11&facets[series][]=MCRFPP21&facets[series][]=MCRFPP31&facets[series][]=MCRFPP41&facets[series][]=MCRFPP51&start={0}&end={1}&sort[0][column]=period&sort[0][direction]=asc&offset=0&length=5000'+'&api_key={2}'

In [5]:
OilProd_qry = requests.get(url_OilProd.format('1980-01','2023-03', api_key))
OilProd_qry.status_code

200

In [6]:
OilProd_qry.json()['response']['total']

2520

In [7]:
df_OilProd = pd.DataFrame(OilProd_qry.json()['response']['data'])

In [8]:
df_OilProd['region']=df_OilProd['series-description'].apply(lambda x: x.split(' (')[0].replace(' ', '_')+'_MBBL' )

In [9]:
df_OilProd['periodDate'] = df_OilProd['period'].apply(lambda x : dt.date.fromisoformat(x+'-01'))

In [10]:
df_OilProd.head(5)

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units,region,periodDate
0,1981-01,R10,PADD 1,EPC0,Crude Oil,FPF,Field Production,MCRFPP11,East Coast (PADD 1) Field Production of Crude ...,4243,MBBL,East_Coast_MBBL,1981-01-01
1,1981-01,R50,PADD 5,EPC0,Crude Oil,FPF,Field Production,MCRFPP51,West Coast (PADD 5) Field Production of Crude ...,81057,MBBL,West_Coast_MBBL,1981-01-01
2,1981-01,R40,PADD 4,EPC0,Crude Oil,FPF,Field Production,MCRFPP41,Rocky Mountain (PADD 4) Field Production of Cr...,18161,MBBL,Rocky_Mountain_MBBL,1981-01-01
3,1981-01,R30,PADD 3,EPC0,Crude Oil,FPF,Field Production,MCRFPP31,Gulf Coast (PADD 3) Field Production of Crude ...,132577,MBBL,Gulf_Coast_MBBL,1981-01-01
4,1981-01,R20,PADD 2,EPC0,Crude Oil,FPF,Field Production,MCRFPP21,Midwest (PADD 2) Field Production of Crude Oil...,28698,MBBL,Midwest_MBBL,1981-01-01


In [11]:
df_OilProd.head(5).to_sql('MetaOilProd', con=engine, if_exists='replace', index=False)

5

In [12]:
engine.connect().execute(text('ALTER TABLE MetaOilProd MODIFY periodDate DATE NOT NULL;'))

In [13]:
engine.connect().execute(text('ALTER TABLE MetaOilProd MODIFY region VARCHAR(21) NOT NULL;'))

In [14]:
engine.connect().execute(text("ALTER TABLE MetaOilProd ADD PRIMARY KEY (periodDate, region);"))

In [15]:
df_OilProd_trab = df_OilProd.pivot(index='periodDate', columns='region', values='value').reset_index()

In [16]:
df_OilProd_trab.head(2)

region,periodDate,East_Coast_MBBL,Gulf_Coast_MBBL,Midwest_MBBL,Rocky_Mountain_MBBL,West_Coast_MBBL
0,1981-01-01,4243,132577,28698,18161,81057
1,1981-02-01,3751,119414,27285,16783,73681


In [17]:
df_OilProd_trab.tail(2)

region,periodDate,East_Coast_MBBL,Gulf_Coast_MBBL,Midwest_MBBL,Rocky_Mountain_MBBL,West_Coast_MBBL
502,2022-11-01,1884,266766,51816,27431,23420
503,2022-12-01,1902,274367,48320,26384,24162


In [48]:
df_OilProd_trab.describe()

region,East_Coast_MBBL,Gulf_Coast_MBBL,Midwest_MBBL,Rocky_Mountain_MBBL,West_Coast_MBBL
count,504.000000,504.000000,504.000000,504.000000,504.000000
mean,1220.408730,126730.404762,28988.617063,15487.119048,57366.484127
std,712.716936,48099.466344,15972.766725,5604.292099,23507.413737
min,399.000000,49933.000000,12123.000000,8139.000000,22409.000000
25%,657.750000,97060.500000,16346.250000,11055.000000,34819.000000
50%,881.000000,103815.000000,22902.500000,14341.500000,54093.500000
75%,1688.000000,132538.750000,35121.000000,18464.250000,81336.750000
max,4243.000000,277358.000000,72803.000000,31796.000000,98056.000000


In [52]:
aux_OilProd = [i for i in df_OilProd_trab.columns] + [len(str(int(df_OilProd_trab.describe().loc['max',][i])))+2 for i in df_OilProd_trab.describe().columns]
aux_OilProd

['periodDate',
 'East_Coast_MBBL',
 'Gulf_Coast_MBBL',
 'Midwest_MBBL',
 'Rocky_Mountain_MBBL',
 'West_Coast_MBBL',
 6,
 8,
 7,
 7,
 7]

In [53]:
crear_OilProd = '''
CREATE TABLE OilProd (
 {0} DATE,
 {1} DECIMAL({6}),
 {2} DECIMAL({7}),
 {3} DECIMAL({8}),
 {4} DECIMAL({9}),
 {5} DECIMAL({10}),
 PRIMARY KEY({0})
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
'''

In [59]:
with engine.connect() as conn:
    sql = text('DROP TABLE IF EXISTS {}'.format('OilProd'))
    conn.execute(sql)
    sql2 = text(crear_OilProd.format(*aux_OilProd))
    conn.execute(sql2)

In [61]:
df_OilProd_trab.to_sql('OilProd', engine, index=False, if_exists='append')

504

## OilPrice & MetaOilPrice

In [63]:
url_OilPrice = 'https://api.eia.gov/v2/petroleum/pri/spt/data/?frequency=monthly&data[0]=value&facets[product][]=EPCWTI&facets[product][]=EPD2DXL0&facets[product][]=EPMRU&facets[product][]=EPCBRENT&facets[product][]=EPLLPA&start={0}&end={1}&sort[0][column]=period&sort[0][direction]=asc&offset=0&length=5000&api_key={2}'

In [66]:
OilPrice_qry = requests.get(url_OilPrice.format('1980-01','2023-03', api_key))
OilPrice_qry.status_code

200

In [67]:
OilPrice_qry.json()['response']['total']

2529

In [68]:
df_OilPrice = pd.DataFrame(OilPrice_qry.json()['response']['data'])

In [72]:
df_OilPrice.tail(7)

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units
2522,2023-02,Y35NY,NEW YORK CITY,EPD2DXL0,No 2 Diesel Low Sulfur (0-15 ppm),PF4,Spot Price FOB,EER_EPD2DXL0_PF4_Y35NY_DPG,New York Harbor Ultra-Low Sulfur No 2 Diesel S...,2.832,$/GAL
2523,2023-02,RGC,NA,EPMRU,Conventional Regular Gasoline,PF4,Spot Price FOB,EER_EPMRU_PF4_RGC_DPG,U.S. Gulf Coast Conventional Gasoline Regular ...,2.416,$/GAL
2524,2023-02,YCUOK,NA,EPCWTI,WTI Crude Oil,PF4,Spot Price FOB,RWTC,"Cushing, OK WTI Spot Price FOB (Dollars per Ba...",76.830,$/BBL
2525,2023-02,ZEU,NA,EPCBRENT,UK Brent Crude Oil,PF4,Spot Price FOB,RBRTE,Europe Brent Spot Price FOB (Dollars per Barrel),82.590,$/BBL
2526,2023-02,Y44MB,NA,EPLLPA,Propane,PF4,Spot Price FOB,EER_EPLLPA_PF4_Y44MB_DPG,"Mont Belvieu, TX Propane Spot Price FOB (Dolla...",0.828,$/GAL
2527,2023-02,RGC,NA,EPD2DXL0,No 2 Diesel Low Sulfur (0-15 ppm),PF4,Spot Price FOB,EER_EPD2DXL0_PF4_RGC_DPG,U.S. Gulf Coast Ultra-Low Sulfur No 2 Diesel S...,2.771,$/GAL
2528,2023-02,Y35NY,NEW YORK CITY,EPMRU,Conventional Regular Gasoline,PF4,Spot Price FOB,EER_EPMRU_PF4_Y35NY_DPG,New York Harbor Conventional Gasoline Regular ...,2.532,$/GAL


In [74]:
prod_OilPrice = {
    'EER_EPD2DXL0_PF4_Y35NY_DPG':'NYH_ULSD_N2_GAL',
    'RWTC':'WTI_FOB_BBL',
    'EER_EPMRU_PF4_RGC_DPG':'USGC_RGAS_GAL',
    'RBRTE':'BRENT_FOB_BBL',
    'EER_EPLLPA_PF4_Y44MB_DPG':'MBTX_C3_GAL',
    'EER_EPMRU_PF4_Y35NY_DPG':'NYH_RGAS_GAL',
    'EER_EPD2DXL0_PF4_RGC_DPG':'USGC_ULSD_N2_GAL'
}

In [75]:
df_OilPrice['prod']=df_OilPrice['series'].map(prod_OilPrice)

In [93]:
df_OilPrice['periodDate'] = df_OilPrice['period'].apply(lambda x: dt.date.fromisoformat(x+'-01'))

In [94]:
df_OilPrice.head(2)

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units,prod,periodDate
0,1986-01,YCUOK,NA,EPCWTI,WTI Crude Oil,PF4,Spot Price FOB,RWTC,"Cushing, OK WTI Spot Price FOB (Dollars per Ba...",22.93,$/BBL,WTI_FOB_BBL,1986-01-01
1,1986-02,YCUOK,NA,EPCWTI,WTI Crude Oil,PF4,Spot Price FOB,RWTC,"Cushing, OK WTI Spot Price FOB (Dollars per Ba...",15.46,$/BBL,WTI_FOB_BBL,1986-02-01


In [95]:
df_OilPrice_trab = df_OilPrice.pivot(index='periodDate', columns='prod', values='value')

In [97]:
df_OilPrice_trab.reset_index(inplace=True)

In [98]:
df_OilPrice_trab.head(3)

prod,periodDate,BRENT_FOB_BBL,MBTX_C3_GAL,NYH_RGAS_GAL,NYH_ULSD_N2_GAL,USGC_RGAS_GAL,USGC_ULSD_N2_GAL,WTI_FOB_BBL
0,1986-01-01,NaN,NaN,NaN,NaN,NaN,NaN,22.93
1,1986-02-01,NaN,NaN,NaN,NaN,NaN,NaN,15.46
2,1986-03-01,NaN,NaN,NaN,NaN,NaN,NaN,12.61


In [99]:
df_OilPrice_trab.tail(3)

prod,periodDate,BRENT_FOB_BBL,MBTX_C3_GAL,NYH_RGAS_GAL,NYH_ULSD_N2_GAL,USGC_RGAS_GAL,USGC_ULSD_N2_GAL,WTI_FOB_BBL
443,2022-12-01,80.92,0.692,2.366,3.117,2.157,2.975,76.44
444,2023-01-01,82.50,0.842,2.593,3.267,2.534,3.207,78.12
445,2023-02-01,82.59,0.828,2.532,2.832,2.416,2.771,76.83


In [100]:
df_OilPrice.tail(7).to_sql('MetaOilPrice', con=engine, if_exists='replace', index=False)

7

In [101]:
df_OilPrice_trab.describe()

prod,BRENT_FOB_BBL,MBTX_C3_GAL,NYH_RGAS_GAL,NYH_ULSD_N2_GAL,USGC_RGAS_GAL,USGC_ULSD_N2_GAL,WTI_FOB_BBL
count,430.000000,369.000000,441.000000,201.000000,441.000000,201.000000,446.000000
mean,48.750140,0.723691,1.356066,2.289627,1.322043,2.231303,46.196099
std,32.935506,0.373357,0.859066,0.757449,0.838910,0.731697,29.517886
min,9.820000,0.209000,0.307000,0.880000,0.294000,0.804000,11.350000
25%,19.035000,0.387000,0.593000,1.794000,0.578000,1.756000,19.975000
50%,39.775000,0.638000,1.119000,2.106000,1.090000,2.071000,36.390000
75%,71.297500,0.980000,1.995000,2.938000,1.958000,2.904000,67.827500
max,132.720000,1.862000,4.082000,4.646000,4.049000,4.286000,133.880000


In [103]:
aux_OilPrice = [i for i in df_OilPrice_trab.columns]
aux_OilPrice

['periodDate',
 'BRENT_FOB_BBL',
 'MBTX_C3_GAL',
 'NYH_RGAS_GAL',
 'NYH_ULSD_N2_GAL',
 'USGC_RGAS_GAL',
 'USGC_ULSD_N2_GAL',
 'WTI_FOB_BBL']

In [104]:
crear_OilPrice = '''
CREATE TABLE OilPrice (
 {0} DATE,
 {1} DECIMAL(7,3),
 {2} DECIMAL(7,3),
 {3} DECIMAL(7,3),
 {4} DECIMAL(7,3),
 {5} DECIMAL(7,3),
 {6} DECIMAL(7,3),
 {7} DECIMAL(7,3),
 PRIMARY KEY({0})
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
'''

In [106]:
with engine.connect() as conn:
    sql = text('DROP TABLE IF EXISTS {}'.format('OilPrice'))
    conn.execute(sql)
    sql2 = text(crear_OilPrice.format(*aux_OilPrice))
    conn.execute(sql2)

In [107]:
df_OilPrice_trab.to_sql('OilPrice', engine, index=False, if_exists='append')

446

## NGasPrice & MetaNGasPrice

In [273]:
url_NGasPrice = 'https://api.eia.gov/v2/natural-gas/pri/fut/data/?frequency=monthly&data[0]=value&facets[series][]=RNGC1&facets[series][]=RNGC2&facets[series][]=RNGC3&facets[series][]=RNGC4&facets[series][]=RNGWHHD&start={0}&end={1}&sort[0][column]=period&sort[0][direction]=asc&offset=0&length=5000'+'&api_key={2}'

In [274]:
NGasPrice_qry = requests.get(url_NGasPrice.format('1986-01','2023-03', api_key))
NGasPrice_qry.status_code

200

In [275]:
NGasPrice_qry.json()['response']['total']

2029

In [276]:
df_NGasPrice = pd.DataFrame( NGasPrice_qry.json()['response']['data'] )

In [277]:
df_NGasPrice[df_NGasPrice['duoarea']=='RGC'].head()

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units
149,1997-01,RGC,NA,EPG0,Natural Gas,PS0,Spot Price,RNGWHHD,Henry Hub Natural Gas Spot Price (Dollars per ...,3.45,$/MMBTU
150,1997-02,RGC,NA,EPG0,Natural Gas,PS0,Spot Price,RNGWHHD,Henry Hub Natural Gas Spot Price (Dollars per ...,2.15,$/MMBTU
155,1997-03,RGC,NA,EPG0,Natural Gas,PS0,Spot Price,RNGWHHD,Henry Hub Natural Gas Spot Price (Dollars per ...,1.89,$/MMBTU
164,1997-04,RGC,NA,EPG0,Natural Gas,PS0,Spot Price,RNGWHHD,Henry Hub Natural Gas Spot Price (Dollars per ...,2.03,$/MMBTU
167,1997-05,RGC,NA,EPG0,Natural Gas,PS0,Spot Price,RNGWHHD,Henry Hub Natural Gas Spot Price (Dollars per ...,2.25,$/MMBTU


In [278]:
df_NGasPrice.iloc[(149-6):].head(7)

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units
143,1996-12,Y35NY,NEW YORK CITY,EPG0,Natural Gas,PE3,Future Contract 3,RNGC3,Natural Gas Futures Contract 3 (Dollars per Mi...,2.817,$/MMBTU
144,1996-12,Y35NY,NEW YORK CITY,EPG0,Natural Gas,PE1,Future Contract 1,RNGC1,Natural Gas Futures Contract 1 (Dollars per Mi...,3.645,$/MMBTU
145,1997-01,Y35NY,NEW YORK CITY,EPG0,Natural Gas,PE1,Future Contract 1,RNGC1,Natural Gas Futures Contract 1 (Dollars per Mi...,3.067,$/MMBTU
146,1997-01,Y35NY,NEW YORK CITY,EPG0,Natural Gas,PE3,Future Contract 3,RNGC3,Natural Gas Futures Contract 3 (Dollars per Mi...,2.393,$/MMBTU
147,1997-01,Y35NY,NEW YORK CITY,EPG0,Natural Gas,PE4,Future Contract 4,RNGC4,Natural Gas Futures Contract 4 (Dollars per Mi...,2.227,$/MMBTU
148,1997-01,Y35NY,NEW YORK CITY,EPG0,Natural Gas,PE2,Future Contract 2,RNGC2,Natural Gas Futures Contract 2 (Dollars per Mi...,2.729,$/MMBTU
149,1997-01,RGC,NA,EPG0,Natural Gas,PS0,Spot Price,RNGWHHD,Henry Hub Natural Gas Spot Price (Dollars per ...,3.450,$/MMBTU


In [279]:
df_NGasPrice = df_NGasPrice.iloc[145:].copy()

In [280]:
series_ngas_price = {
    'RNGC2':'NYMEX_NG_FUT2_MMBTU', 
    'RNGC1':'NYMEX_NG_FUT1_MMBTU',
    'RNGC3':'NYMEX_NG_FUT3_MMBTU',
    'RNGC4':'NYMEX_NG_FUT4_MMBTU',
    'RNGWHHD':'HH_NG_SPOT_MMBTU'
}

In [281]:
df_NGasPrice['prod'] = df_NGasPrice['series'].map(series_ngas_price)

In [282]:
df_NGasPrice['periodDate'] = df_NGasPrice['period'].apply(lambda x : dt.date.fromisoformat(x+'-01'))

In [283]:
df_NGasPrice[df_NGasPrice['period']=='2022-12']

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units,prod,periodDate
1700,2022-12,Y35NY,NEW YORK CITY,EPG0,Natural Gas,PE1,Future Contract 1,RNGC1,Natural Gas Futures Contract 1 (Dollars per Mi...,5.77,$/MMBTU,NYMEX_NG_FUT1_MMBTU,2022-12-01
1701,2022-12,Y35NY,NEW YORK CITY,EPG0,Natural Gas,PE2,Future Contract 2,RNGC2,Natural Gas Futures Contract 2 (Dollars per Mi...,5.60,$/MMBTU,NYMEX_NG_FUT2_MMBTU,2022-12-01
1702,2022-12,Y35NY,NEW YORK CITY,EPG0,Natural Gas,PE3,Future Contract 3,RNGC3,Natural Gas Futures Contract 3 (Dollars per Mi...,5.09,$/MMBTU,NYMEX_NG_FUT3_MMBTU,2022-12-01
1703,2022-12,RGC,NA,EPG0,Natural Gas,PS0,Spot Price,RNGWHHD,Henry Hub Natural Gas Spot Price (Dollars per ...,5.53,$/MMBTU,HH_NG_SPOT_MMBTU,2022-12-01
1704,2022-12,Y35NY,NEW YORK CITY,EPG0,Natural Gas,PE4,Future Contract 4,RNGC4,Natural Gas Futures Contract 4 (Dollars per Mi...,4.67,$/MMBTU,NYMEX_NG_FUT4_MMBTU,2022-12-01


In [286]:
df_NGasPrice[df_NGasPrice['period']=='2022-12'].to_sql('MetaNGasPrice', con=engine, if_exists='replace', index=False)

5

In [287]:
df_NGasPrice_trab = df_NGasPrice.pivot(index='periodDate', values = 'value', columns = 'prod')

In [288]:
df_NGasPrice_trab.head(2)

prod,HH_NG_SPOT_MMBTU,NYMEX_NG_FUT1_MMBTU,NYMEX_NG_FUT2_MMBTU,NYMEX_NG_FUT3_MMBTU,NYMEX_NG_FUT4_MMBTU
periodDate,,,,,
1997-01-01,3.45,3.067,2.729,2.393,2.227
1997-02-01,2.15,2.065,2.016,1.995,1.999


In [289]:
df_NGasPrice_trab.tail(2)

prod,HH_NG_SPOT_MMBTU,NYMEX_NG_FUT1_MMBTU,NYMEX_NG_FUT2_MMBTU,NYMEX_NG_FUT3_MMBTU,NYMEX_NG_FUT4_MMBTU
periodDate,,,,,
2023-01-01,3.27,3.42,3.20,3.17,3.25
2023-02-01,2.38,2.44,2.53,2.68,2.87


In [290]:
df_NGasPrice_trab.describe()

prod,HH_NG_SPOT_MMBTU,NYMEX_NG_FUT1_MMBTU,NYMEX_NG_FUT2_MMBTU,NYMEX_NG_FUT3_MMBTU,NYMEX_NG_FUT4_MMBTU
count,314.000000,314.000000,314.000000,314.000000,314.000000
mean,4.248694,4.295213,4.385191,4.453427,4.492115
std,2.163350,2.202880,2.247150,2.280323,2.279136
min,1.630000,1.700000,1.780000,1.820000,1.850000
25%,2.662500,2.702750,2.760000,2.803000,2.850000
50%,3.620000,3.621000,3.789500,3.846000,3.869500
75%,5.385000,5.361750,5.540750,5.451500,5.406250
max,13.420000,13.454000,13.827000,14.178000,14.010000


In [291]:
df_NGasPrice_trab.reset_index(inplace=True)

In [292]:
aux_NGasPrice =[i for i in df_NGasPrice_trab.columns]
aux_NGasPrice

['periodDate',
 'HH_NG_SPOT_MMBTU',
 'NYMEX_NG_FUT1_MMBTU',
 'NYMEX_NG_FUT2_MMBTU',
 'NYMEX_NG_FUT3_MMBTU',
 'NYMEX_NG_FUT4_MMBTU']

In [293]:
crear_NGasPrice = '''
CREATE TABLE NGasPrice (
 {0} DATE,
 {1} DECIMAL(7,3),
 {2} DECIMAL(7,3),
 {3} DECIMAL(7,3),
 {4} DECIMAL(7,3),
 {5} DECIMAL(7,3),
 PRIMARY KEY({0})
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
'''

In [294]:
with engine.connect() as conn:
    sql = text('DROP TABLE IF EXISTS {}'.format('NGasPrice'))
    conn.execute(sql)
    sql2 = text(crear_NGasPrice.format(*aux_NGasPrice))
    conn.execute(sql2)

In [295]:
df_NGasPrice_trab.to_sql('NGasPrice', engine, index=False, if_exists='append')

314

## NGasStorage & MetaNGasStorage

In [180]:
url_NGasStorage = 'https://api.eia.gov/v2/natural-gas/stor/sum/data/?frequency=monthly&data[0]=value&facets[process][]=SAT&facets[series][]=NGM_EPG0_SAT_R48_MMCF&start={0}&end={1}&sort[0][column]=period&sort[0][direction]=asc&offset=0&length=5000'+'&api_key={2}'

In [181]:
NGasStorage_qry = requests.get(url_NGasStorage.format('1997-01','2023-03', api_key))
NGasStorage_qry.status_code

200

In [182]:
NGasStorage_qry.json()['response']['total']

144

In [184]:
df_NGasStorage = pd.DataFrame(NGasStorage_qry.json()['response']['data'])

In [189]:
df_NGasStorage.head(2)

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units
0,2011-01,R48,NA,EPG0,Natural Gas,SAT,Total Underground Storage,NGM_EPG0_SAT_R48_MMCF,Lower 48 States Natural Gas Underground Storag...,6608635,MMCF
1,2011-02,R48,NA,EPG0,Natural Gas,SAT,Total Underground Storage,NGM_EPG0_SAT_R48_MMCF,Lower 48 States Natural Gas Underground Storag...,6024215,MMCF


In [191]:
df_NGasStorage['periodDate'] = df_NGasStorage['period'].apply(lambda x: dt.date.fromisoformat(x+'-01'))

In [193]:
#this is just for the meta: 
df_NGasStorage['value_renaming'] = 'LOWER48_NATGAS_STORAGE_MMCF'

In [197]:
df_NGasStorage.head(1).to_sql('MetaNGasStorage', con=engine, if_exists='replace', index=False)
df_NGasStorage.head(1)

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units,periodDate,value_renaming
0,2011-01,R48,NA,EPG0,Natural Gas,SAT,Total Underground Storage,NGM_EPG0_SAT_R48_MMCF,Lower 48 States Natural Gas Underground Storag...,6608635,MMCF,2011-01-01,LOWER48_NATGAS_STORAGE_MMCF


In [209]:
df_NGasStorage_trab = df_NGasStorage[['periodDate','value']].copy()

In [210]:
df_NGasStorage_trab.rename(columns={'value':'LOWER48_NATGAS_STORAGE_MMCF'}, inplace=True)

In [ ]:
df_NGasStorage_trab.

In [211]:
df_NGasStorage_trab.head(2)

,periodDate,LOWER48_NATGAS_STORAGE_MMCF
0,2011-01-01,6608635
1,2011-02-01,6024215


In [212]:
df_NGasStorage_trab.describe()

,LOWER48_NATGAS_STORAGE_MMCF
count,1.440000e+02
mean,7.079872e+06
std,7.632040e+05
min,5.172535e+06
25%,6.499054e+06
50%,7.118624e+06
75%,7.702902e+06
max,8.334627e+06


In [205]:
crear_NGasStorage = '''
CREATE TABLE NGasStorage ( 
periodDate DATE NOT NULL, 
LOWER48_NATGAS_STORAGE_MMCF DECIMAL(9), 
PRIMARY KEY(periodDate))
ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
'''

In [206]:
with engine.connect() as conn:
    sql = text('DROP TABLE IF EXISTS {}'.format('NGasStorage'))
    conn.execute(sql)
    sql2 = text(crear_NGasStorage)
    conn.execute(sql2)

In [213]:
df_NGasStorage_trab.to_sql('NGasStorage', engine, index=False, if_exists='append')

144

## DryGasProd & MetaDryGasProd

In [214]:
url_DryGasProd = 'https://api.eia.gov/v2/natural-gas/prod/sum/data/?frequency=monthly&data[0]=value&facets[process][]=FPD&facets[series][]=N9070US2&start={}&end={}&sort[0][column]=period&sort[0][direction]=asc&offset=0&length=5000'+'&api_key={}'

In [217]:
DryGasProd_qry = requests.get(url_DryGasProd.format('1997-01','2023-03', api_key))
DryGasProd_qry.status_code

200

In [218]:
df_DryGasProd = pd.DataFrame(DryGasProd_qry.json()['response']['data'])

In [221]:
df_DryGasProd.sample(5)

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units
38,2000-03,NUS,U.S.,EPG0,Natural Gas,FPD,Dry Production,N9070US2,U.S. Dry Natural Gas Production (MMcf),1635707,MMCF
221,2015-06,NUS,U.S.,EPG0,Natural Gas,FPD,Dry Production,N9070US2,U.S. Dry Natural Gas Production (MMcf),2218529,MMCF
136,2008-05,NUS,U.S.,EPG0,Natural Gas,FPD,Dry Production,N9070US2,U.S. Dry Natural Gas Production (MMcf),1716720,MMCF
238,2016-11,NUS,U.S.,EPG0,Natural Gas,FPD,Dry Production,N9070US2,U.S. Dry Natural Gas Production (MMcf),2160600,MMCF
74,2003-03,NUS,U.S.,EPG0,Natural Gas,FPD,Dry Production,N9070US2,U.S. Dry Natural Gas Production (MMcf),1662980,MMCF


In [222]:
df_DryGasProd['periodDate'] = df_DryGasProd['period'].apply(lambda x: dt.date.fromisoformat(x+'-01'))

In [223]:
#Again, this is just for the meta: 
df_DryGasProd['value_renaming'] = 'DRYNATGAS_PROD_MMCF'

In [224]:
df_DryGasProd.head(1).to_sql('MetaDryGasProd', con=engine, if_exists='replace', index=False)
df_DryGasProd.head(1)

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units,periodDate,value_renaming
0,1997-01,NUS,U.S.,EPG0,Natural Gas,FPD,Dry Production,N9070US2,U.S. Dry Natural Gas Production (MMcf),1617923,MMCF,1997-01-01,DRYNATGAS_PROD_MMCF


In [225]:
df_DryGasProd_trab = df_DryGasProd[['periodDate','value']].copy()

In [226]:
df_DryGasProd_trab.rename(columns={'value':'DRYNATGAS_PROD_MMCF'}, inplace=True)

In [228]:
df_DryGasProd_trab.sample(2)

,periodDate,DRYNATGAS_PROD_MMCF
175,2011-08-01,1959548
124,2007-05-01,1643202


In [229]:
df_DryGasProd_trab.describe()

,DRYNATGAS_PROD_MMCF
count,3.130000e+02
mean,1.979829e+06
std,4.837422e+05
min,1.336065e+06
25%,1.593864e+06
50%,1.755132e+06
75%,2.266298e+06
max,3.145142e+06


In [231]:
crear_DRYNATGAS_PROD_MMCF = '''
CREATE TABLE DryGasProd ( 
periodDate DATE NOT NULL, 
DRYNATGAS_PROD_MMCF DECIMAL(9), 
PRIMARY KEY(periodDate))
ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
'''

with engine.connect() as conn:
    sql = text('DROP TABLE IF EXISTS {}'.format('DryGasProd'))
    conn.execute(sql)
    sql2 = text(crear_DRYNATGAS_PROD_MMCF)
    conn.execute(sql2)

df_DryGasProd_trab.to_sql('DryGasProd', engine, index=False, if_exists='append')

313

## STEO and MetaSTEO

In [234]:
url_STEO = steo_url = 'https://api.eia.gov/v2/steo/data/?frequency=monthly&data[0]=value&facets[seriesId][]=NGHHUUS&facets[seriesId][]=WTIPUUS&sort[0][column]=period&sort[0][direction]=asc&offset=0&length=5000&start={}'+'&api_key={}'

In [235]:
STEO_qry = requests.get(url_STEO.format('2022-02', api_key))
STEO_qry.status_code

200

In [236]:
df_STEO = pd.DataFrame(STEO_qry.json()['response']['data'])

In [238]:
df_STEO.tail(2)

,period,seriesId,seriesDescription,value,unit
68,2024-12,NGHHUUS,Natural Gas Henry Hub Spot Price ($/mmBtu),4.245872,dollars per million Btu
69,2024-12,WTIPUUS,West Texas Intermediate Crude Oil Price,69.000000,dollars per barrel


In [239]:
df_STEO['periodDate'] = df_STEO['period'].apply(lambda x : dt.date.fromisoformat(x+'-01'))

In [240]:
df_STEO['seriesId'].unique()

array(['NGHHUUS', 'WTIPUUS'], dtype=object)

In [241]:
prod_from_seriesId = {
    'NGHHUUS':'HH_NG_PROY_MMBTU',
    'WTIPUUS':'WTI_PROY_BBL'
}
df_STEO['prod'] = df_STEO['seriesId'].map(prod_from_seriesId)

In [253]:
df_STEO['updateDate'] = dt.date.today()

In [254]:
df_STEO['periodDate'] = df_STEO['period'].apply(lambda x : dt.date.fromisoformat(x+'-01'))

In [255]:
df_STEO.tail(2).to_sql('MetaSTEO', con=engine, if_exists='replace', index=False)
df_STEO.tail(2)

,period,seriesId,seriesDescription,value,unit,periodDate,prod,updateDate
68,2024-12,NGHHUUS,Natural Gas Henry Hub Spot Price ($/mmBtu),4.245872,dollars per million Btu,2024-12-01,HH_NG_PROY_MMBTU,2023-03-31
69,2024-12,WTIPUUS,West Texas Intermediate Crude Oil Price,69.000000,dollars per barrel,2024-12-01,WTI_PROY_BBL,2023-03-31


In [261]:
df_STEO_trab = df_STEO.pivot(index='periodDate', columns='prod', values='value')

In [262]:
df_STEO_trab['updateDate'] = dt.date.today()

In [263]:
df_STEO_trab.reset_index(inplace=True)

In [266]:
df_STEO_trab.tail(2)

prod,periodDate,HH_NG_PROY_MMBTU,WTI_PROY_BBL,updateDate
33,2024-11-01,4.026826,69.0,2023-03-31
34,2024-12-01,4.245872,69.0,2023-03-31


In [265]:
df_STEO_trab.describe()

prod,HH_NG_PROY_MMBTU,WTI_PROY_BBL
count,35.000000,35.000000
mean,4.444485,81.090571
std,1.731391,12.413591
min,2.380000,69.000000
25%,3.257562,73.500000
50%,3.833210,76.830000
75%,5.175000,84.315000
max,8.800000,114.840000


In [269]:
crear_STEO = '''
CREATE TABLE STEO ( 
periodDate DATE NOT NULL,
updateDate DATE NOT NULL,
HH_NG_PROY_MMBTU DECIMAL(10,6),
WTI_PROY_BBL DECIMAL(11,6),
PRIMARY KEY(periodDate, updateDate))
ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
'''

with engine.connect() as conn:
    sql = text('DROP TABLE IF EXISTS {}'.format('STEO'))
    conn.execute(sql)
    sql2 = text(crear_STEO)
    conn.execute(sql2)

df_STEO_trab.to_sql('STEO', engine, index=False, if_exists='append')

35

## APIStrings

In [309]:
lines =[
    {'dbtable':'OilProd', 'strings': url_OilProd },
    {'dbtable':'OilPrice', 'strings': url_OilPrice },
    {'dbtable':'NGasPrice', 'strings': url_NGasPrice },
    {'dbtable':'NGasStorage', 'strings': url_NGasStorage },
    {'dbtable':'DryGasProd', 'strings': url_DryGasProd },
    {'dbtable':'STEO', 'strings': url_STEO }
]

In [310]:
df_APIStrings = pd.DataFrame(lines)

In [311]:
df_APIStrings

,dbtable,strings
0,OilProd,https://api.eia.gov/v2/petroleum/crd/crpdn/dat...
1,OilPrice,https://api.eia.gov/v2/petroleum/pri/spt/data/...
2,NGasPrice,https://api.eia.gov/v2/natural-gas/pri/fut/dat...
3,NGasStorage,https://api.eia.gov/v2/natural-gas/stor/sum/da...
4,DryGasProd,https://api.eia.gov/v2/natural-gas/prod/sum/da...
5,STEO,https://api.eia.gov/v2/steo/data/?frequency=mo...


In [340]:
df_APIStrings['lendbtable'] = df_APIStrings['dbtable'].apply(lambda x : len(x))
df_APIStrings['lenstrings'] = df_APIStrings['strings'].apply(lambda x : len(x))

In [341]:
df_APIStrings

,dbtable,strings,lendbtable,lenstrings
0,OilProd,https://api.eia.gov/v2/petroleum/crd/crpdn/dat...,7,307
1,OilPrice,https://api.eia.gov/v2/petroleum/pri/spt/data/...,8,303
2,NGasPrice,https://api.eia.gov/v2/natural-gas/pri/fut/dat...,9,294
3,NGasStorage,https://api.eia.gov/v2/natural-gas/stor/sum/da...,11,239
4,DryGasProd,https://api.eia.gov/v2/natural-gas/prod/sum/da...,10,223
5,STEO,https://api.eia.gov/v2/steo/data/?frequency=mo...,4,206


In [342]:
crear_API = '''
CREATE TABLE APIStrings ( 
dbtable VARCHAR(11) NOT NULL, 
strings VARCHAR(310), 
PRIMARY KEY(dbtable))
ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
'''

with engine.connect() as conn:
    sql = text('DROP TABLE IF EXISTS {}'.format('APIStrings'))
    conn.execute(sql)
    sql2 = text(crear_API)
    conn.execute(sql2)

df_APIStrings[['dbtable', 'strings']].to_sql('APIStrings', engine, index=False, if_exists='append')

6

##### Just a quick validation:

In [343]:
with engine.connect() as conn:
    qry = text('SELECT * FROM APIStrings')
    res = conn.execute(qry)

In [344]:
resp = res.all()

In [346]:
df_api = pd.DataFrame(resp)

In [357]:
df_api[df_api['dbtable'] == 'OilProd']['strings'].values[0]

'https://api.eia.gov/v2/petroleum/crd/crpdn/data/?frequency=monthly&data[0]=value&facets[series][]=MCRFPP11&facets[series][]=MCRFPP21&facets[series][]=MCRFPP31&facets[series][]=MCRFPP41&facets[series][]=MCRFPP51&start={0}&end={1}&sort[0][column]=period&sort[0][direction]=asc&offset=0&length=5000&api_key={2}'

In [358]:
url_OilProd

'https://api.eia.gov/v2/petroleum/crd/crpdn/data/?frequency=monthly&data[0]=value&facets[series][]=MCRFPP11&facets[series][]=MCRFPP21&facets[series][]=MCRFPP31&facets[series][]=MCRFPP41&facets[series][]=MCRFPP51&start={0}&end={1}&sort[0][column]=period&sort[0][direction]=asc&offset=0&length=5000&api_key={2}'

In [360]:
df_api[df_api['dbtable'] == 'OilProd']['strings'].values[0] == url_OilProd

True

In [359]:
engine.dispose()